In [29]:
import math
import os
import shutil
import altair as alt
import numpy as np
import pandas as pd
import scipy.stats
import Bio.SeqIO
import yaml
import csv
import subprocess
import pandas as pd

In [30]:

# Load the CSV file
df = pd.read_csv('../../results/annotated_summary_csvs/entry_293T-Mxra8_C636_293T-TIM1_Mxra8-binding_annotated_site_means.csv')

# Select and clean relevant columns
columns_to_keep = ['protein_site', 'entry in 293T_Mxra8 cells', '6nk7_chain']
new_df = df[columns_to_keep].copy()

# Convert 'protein_site' to numeric, drop NaNs, and cast to int
new_df['protein_site'] = pd.to_numeric(new_df['protein_site'], errors='coerce')
new_df = new_df.dropna(subset=['protein_site'])
new_df['protein_site'] = new_df['protein_site'].astype(int)

# Save the cleaned DataFrame
output_path = '../../manual_analyses/chimera_structures/results/293T-Mxra8_entry_func_effects_mean.csv'

new_df.to_csv(output_path, index=False)

In [31]:
# prepare data in chimera format
def aggregate_entry_mean(infile, name, outfile):
    tmp_df = pd.read_csv(infile)
    tmp_df = tmp_df.round(3)

    # Modify the dataframe to prepend a tab character and format as strings
    tmp_df["protein_site"] = tmp_df["protein_site"].astype(str)
    tmp_df["entry in 293T_Mxra8 cells"] = tmp_df["entry in 293T_Mxra8 cells"].astype(str)
    tmp_df["formatted"] = "\t" + "/"  + tmp_df['6nk7_chain'] + ":" + tmp_df["protein_site"] + "\t" + tmp_df["entry in 293T_Mxra8 cells"]

    with open(outfile, "w") as f:
        # Write header lines
        f.write(f"attribute: {name}\n")
        f.write("match mode: any\n")
        f.write("recipient: residues\n")

    # Append the tab separated dataframe to the file without quotes and with an escape character
    tmp_df["formatted"].to_csv(outfile, sep="\t", index=False, header=False, mode="a")
    subprocess.run(["sed", "-i", 's/"//g', outfile], check=True)

# Call function above
aggregate_entry_mean('../../manual_analyses/chimera_structures/results/293T-Mxra8_entry_func_effects_mean.csv', "entry", '../../manual_analyses/chimera_structures/results/293T-Mxra8_entry_func_effects.defattr')
